In [1]:
import os
import numpy as np
import pandas as pd
import pynndescent
import duckdb
import pickle
import scipy.sparse
import importlib

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
HASHED_FEATURES        = os.environ["HASHED_FEATURES"]
HASHED_FEATURES_IDX    = os.environ["HASHED_FEATURES_IDX"]
KNN_RANDOM_STATE       = int(os.environ["KNN_RANDOM_STATE"])
KNN_METRIC             = os.environ["KNN_METRIC"]
KNN_NEIGHBORS          = int(os.environ["KNN_NEIGHBORS"])
KNN_INDEX              = os.environ["KNN_INDEX"]

print(f"""
HASHED_FEATURES        = {HASHED_FEATURES}
HASHED_FEATURES_IDX    = {HASHED_FEATURES_IDX}
KNN_RANDOM_STATE       = {KNN_RANDOM_STATE}
KNN_METRIC             = {KNN_METRIC}
KNN_NEIGHBORS          = {KNN_NEIGHBORS}
KNN_INDEX              = {KNN_INDEX}
""")


HASHED_FEATURES        = ./DATA/hashed_features.dat
HASHED_FEATURES_IDX    = ./DATA/hashed_features_idx.parquet
KNN_RANDOM_STATE       = 42
KNN_METRIC             = cosine
KNN_NEIGHBORS          = 250
KNN_INDEX              = ./DATA/knn_index.pickle



Os dados gerados pelo notebook anterior são lidos abaixo

In [3]:
hashed_features = scipy.sparse.load_npz(HASHED_FEATURES + ".npz").tocsr()
hashed_features.shape

(86332, 500000)

In [4]:
type(hashed_features)

scipy.sparse._csr.csr_matrix

In [5]:
hashed_features_idx = pd.read_parquet(HASHED_FEATURES_IDX)
hashed_features_idx.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86332 entries, 0 to 86331
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          86332 non-null  int64 
 1   chave_usuario  86300 non-null  object
 2   lotacao_topo   86332 non-null  object
 3   sigla_lotacao  86332 non-null  object
 4   tipo_usuario   86332 non-null  object
 5   cargo          86332 non-null  object
 6   enfase         86332 non-null  object
 7   funcao         86332 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.3+ MB


A biblioteca PyNNDescent é utilizada na classe abaixo para indexar a matriz esparsa de usuários x hashed features.

Como esta funcionalidade precisará ser reutilizada no recomendador, estamos escrevendo a mesma em uma classe separada para possibilitar posterior reuso.

In [6]:
%%writefile knn_indexer.py
import numpy as np
import pickle
import pynndescent

class KnnIndex:
    
    def __init__(self, metric, n_neighbors, random_state):
        self.metric       = metric
        self.n_neighbors  = n_neighbors
        self.random_state = random_state
        self.data         = None
        self.neighbors_of = None
        self.distances_of = None

    def index(self, data):
        knn_index = pynndescent.NNDescent(
            data         = data
        ,   metric       = self.metric
        ,   n_neighbors  = self.n_neighbors
        ,   random_state = self.random_state
        ,   low_memory   = True
        ,   verbose      = True
        ,   compressed   = False # neighbor_graph is deleted when the index is compressed
        )
        neighbors, distances = knn_index.neighbor_graph
        self.data            = data
        self.neighbors_of    = neighbors.astype("int32", copy=False)
        self.distances_of    = distances.astype("float32", copy=False)

    def save(self, filename):
        with open(filename, "wb") as fh:
            pickle.dump(self, fh)
            
    def get_row(self, idx):
        assert self.data is not None
        return self.data[idx]
    
    def get_neighbors(self, idx):
        assert self.data is not None
        neighbors = self.neighbors_of[idx]
        distances = self.distances_of[idx]
        return neighbors, distances
    
    @classmethod
    def load(klass, filename):
        with open(filename, "rb") as fh:
            return pickle.load(fh)

Overwriting knn_indexer.py


O código abaixo serve para recarregar o módulo em que a classe **KnnIndex** é criada.

Por padrão um módulo não é reimportado caso uma instrução import seja repetida

In [7]:
import knn_indexer
importlib.reload(knn_indexer)
from knn_indexer import KnnIndex

O método abaixo indexa os 85.000+ usuários indicando quais são os KNN_NEIGHBORS mais próximos utilizando a biblioteca PyNNDescent.

Esta é a parte mais lenta do processamento da solução, apesar do uso de múltiplas CPU's.

Dado a elevada dimensionalidade do conjunto de dados somada ao uso de KNN_NEIGHBORS(atualmente 250), a biblioteca PyNNDescent não consegue criar o grafo de vizinhos que possibilita o cômputo dinâmico do KNN aproximadO (grafo criado implicitamente na primeira query ou quando o método *prepare* é chamado). Ao tentar criar o grafo, o consumo de memória excede 20+ gigas e ocasiona no término do processo.

Esta limitação indica que o recomendador não funciona para usuários não presentes no conjunto de dados processado.

Para que seja possível recomendar perfis para usuários ainda não vistos, seria preciso avaliar algumas das possibilidades abaixo:

 1 - Entender a causa da explosão do consumo de memória dentro da biblioteca PyNNDescent. Potencialmente sendo causada pela  convertão (possivelmente implícita) de linhas da matriz esparsa (que hoje utiliza 500.000 colunas) em linhas densas.
 
 2 - Utilizar alguma técnica de redução de dimensionalidade, como por exemplo o refatoramento da matriz usando **Singular Value Decomposition** ou **Non-Negative Matrix Factorization**.
 
 3 - Reduzir o conjunto de dados global por múltiplos conjuntos de dados por lotação topo.
 
 Estes 3 pontos, entretanto, encontram-se fora do escopo desse trabalho

In [ ]:
knn_index = KnnIndex(
    metric       = KNN_METRIC
,   n_neighbors  = KNN_NEIGHBORS
,   random_state = KNN_RANDOM_STATE
)
knn_index.index(hashed_features)

Sun Jul 16 16:29:31 2023 Building RP forest with 22 trees
Sun Jul 16 16:29:43 2023 metric NN descent for 16 iterations


Uma vez calculado os K vizinhos mais próximos, é possível obter a lista de índices dos vizinhos assim como suas respectivas distâncias. O arquivo parquet HASHED_FEATURES_IDX contém os índices das linhas correspondentes a cada usuário no conjunto de dados processado.

Atualmente a métrica de distância utilizada é a distância do cosseno baseada na similaridade do cosseno (1 - similaridade). Esta distância calcula basicamente o ângulo em espaço n-vetorial que dois vetores formam entre si, sendo que quanto menor a distância mais próximos estes vetores se encontram. A distância varia entre 0.0 e 1.0 e este intervalo limitado é utilizado posteriormente na fórmula de *scoring* dos perfis recomendados

In [11]:
neighbors, distances = knn_index.get_neighbors(0)
display(neighbors)
display(distances)

array([11080,     0, 59152, 51250, 24581, 24742, 58253,  5381, 29334,
       19220, 81875,  5129, 64794, 59437, 28229,   967, 80180, 58105,
       27382, 26544, 58198, 38320, 27953, 56639, 10148, 14332, 25573,
         908, 26130, 64954, 25287, 58041, 26495, 15523, 28842, 26753,
        5354, 63230,  5418,  9597, 32426, 57225, 28978, 32496, 13568,
        8499, 19391, 25197, 71572, 14190, 33207,  3496, 33537, 52526,
       33525, 33464, 58827,  8329, 53906, 30310, 26543,  1146, 31718,
        5602, 25909, 32453, 27874,  1514, 80161, 63641, 33206, 18867,
       27564, 27567, 54165, 23454, 12672, 27360,  1602,  7567, 18159,
       38197, 68702, 26431, 25670, 65608, 58165, 29158, 24517,  2119,
        2101,  8208, 54040, 25559, 33876, 38327, 80129, 38239, 11533,
        2113, 26032, 17764, 31191,  2108, 81978, 26091, 71450, 37677,
       15539, 85250, 73627, 58949, 54850,  7533, 32389,  2780, 45425,
       45592, 25760,  2102, 16067,  2791,  7597, 36230, 45589, 27496,
       31234, 31198,

array([1.19209268e-07, 1.19209268e-07, 1.09134745e-02, 1.09134745e-02,
       1.09134745e-02, 1.09134745e-02, 1.09134745e-02, 1.09134745e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       1.11570377e-02, 1.11570377e-02, 1.11570377e-02, 1.11570377e-02,
       2.14771815e-02, 2.14771815e-02, 2.21894905e-02, 2.21894905e-02,
       2.21894905e-02, 2.21894905e-02, 2.21894905e-02, 2.21894905e-02,
       2.21894905e-02, 2.21894905e-02, 2.21894905e-02, 2.21894905e-02,
       2.21894905e-02, 2.21894905e-02, 2.21894905e-02, 2.21894905e-02,
       2.21894905e-02, 2.21894905e-02, 3.28606367e-02, 3.28606367e-02,
      

O índice, uma vez calculado, é salvo para poder ser consultado posteriormente.

In [13]:
knn_index.save(KNN_INDEX)
print(f"knn index has been built -> {KNN_INDEX}")

knn index has been built -> ./DATA/knn_index.pickle
